* 하나의 검색 키워드에 대해서만 자동화(어러 종류의 주제를 한번에 크롤링 할 수 있게)

# import module

In [1]:
import pandas as pd
import numpy as np
import time
import re
import os
import json
from random import random
from selenium.webdriver import Chrome
from urllib.request import urlretrieve
from uuid import uuid4
from io import BytesIO

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import urllib.request

# 창 띄우기

In [2]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager

# def set_chrome_driver():
#     chrome_options = webdriver.ChromeOptions()
#     driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
#     return driver

In [3]:
def open_browser():
    browser = Chrome("chromedriver.exe") # 입력
    browser.get("https://www.instagram.com/accounts/login/?hl=ko&source=auth_switcher")
    return browser

# Login

In [8]:
def login(browser):
    id="dg1dgu" # 입력
    pwd="DGU1dgu1" # 입력
    input_id=browser.find_element(By.XPATH,'//*[@id="loginForm"]/div/div[1]/div/label/input') #아이디 입럭 객체 선택
    input_pwd=browser.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[2]/div/label/input') #패스워드 입력 객체 선택
    input_id.send_keys(id)
    input_pwd.send_keys(pwd)
    time.sleep(1)
    
    browser.find_element(By.XPATH,'//*[@id="loginForm"]/div/div[3]/button/div').click() #로그인 버튼 클릭
    time.sleep(5)
    
    browser.find_element(By.XPATH,'//*[@id="react-root"]/section/main/div/div/div/div/button').click() #로그인정보 저장 나중에하기 버튼
    

# 검색키워드 검색

In [10]:
def search_keyword(browser, keyword):
    url = f"https://www.instagram.com/explore/tags/{keyword}/"
    browser.get(url)

# 첫번째 게시글 오픈

In [14]:
def open_first_post(browser):
    first_post = browser.find_elements_by_class_name("_9AhH0")[0]
    print(type(first_post))
    print(first_post)
    browser.execute_script("arguments[0].click();",first_post)

# 좋아요 개수 가져오기(넘기면서 반복)

In [19]:
def get_likes(browser):
    likes=0
    try:
        like = browser.find_elements_by_xpath('/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/div/a/div/span')
        likes = int(like[0].text)
        if likes > 30:
            return likes
    except Exception as ex:
        print("get_likes 에러발생", ex)
        return likes

# 이미지 URL(넘기면서 반복)

* https://mizykk.tistory.com/55
* openCV를 이용해서 이미지 중복을 처리할 수 있을 듯.

In [119]:
'''
# 앞-현재-뒤 이렇게 컨텐츠가 넘어옴
img = browser.find_element_by_class_name("RnEpo")
img = img.find_elements_by_class_name('KL4Bh') # 동영상이면 
img_url = img[0].find_element_by_tag_name('img').get_attribute("src") #동영상이면 list index out of range 에러뜸
img_url
'''

'\n# 앞-현재-뒤 이렇게 컨텐츠가 넘어옴\nimg = browser.find_element_by_class_name("RnEpo")\nimg = img.find_elements_by_class_name(\'KL4Bh\') # 동영상이면 \nimg_url = img[0].find_element_by_tag_name(\'img\').get_attribute("src") #동영상이면 list index out of range 에러뜸\nimg_url\n'

In [22]:
def get_image(browser, keyword):
    img_name = "NaN"
    
    try:
        #img = browser.find_elements_by_class_name("KL4Bh") #이렇게 되면 마지막 놈이 뜸. 동영상은 처리 못함.
        img = browser.find_element_by_class_name("RnEpo")
        img = img.find_elements_by_class_name('KL4Bh') # 동영상이면 
        #img_url = img[-1].find_element_by_css_selector("article img").get_attribute("src")
        img_url = img[0].find_element_by_tag_name('img').get_attribute("src")

        uuid =uuid4() #유일한 이름 할당. 중복
        #urlretrieve(img_url, f"data/{keyword}/{uuid}.jpg") #이건 다운로드
        img_name = f"{uuid}.jpg"
        return img_name, img_url
    
    except Exception as ex:
        #에러가 발생하면 img_name이 "NaN"으로 출력됨.
        print("get_image 에러발생", ex)
        return img_name, img_url

# 이미지 저장(넘기면서 반복)

In [122]:
def save_image(browser, keyword, img_name, img_url):
    urlretrieve(img_url, f"data/{keyword}/{img_name}")

# 해시태그 출력(NaN 확인 필요) (넘기면서 반복)

In [156]:
def get_hashtag(browser):
    components = browser.find_elements_by_class_name("xil3i")
    #components = browser.find_elements(By.CSS_SELECTOR, "xil3i") #안되는건가?
    if len(components) != 0:
        
        try:
            all_hashtags = [component.text for component in components]
            return list(set(all_hashtags))
        
        except Exception as ex:
            print("get_hashtags 에러발생", ex)
            return []

In [158]:
get_hashtag(browser)

C:\Users\User\AppData\Local\Temp\ipykernel_16500\885970769.py:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  components = browser.find_elements_by_class_name("xil3i")


['#소문난암뽕수육',
 '#음식',
 '#경주',
 '#데일리',
 '#일상소통',
 '#데일리그램',
 '#술',
 '#먹스타',
 '#술스타그램',
 '#먹방스타그램',
 '#맛있다그램',
 '#먹팔',
 '#친절',
 '#좋아요',
 '#홈쿡',
 '#음식스타그램',
 '#일상기록',
 '#맛집스타그램',
 '#팔로워',
 '#선팔',
 '#일상스타그램',
 '#요리스타그램',
 '#소통',
 '#먹방',
 '#맞팔',
 '#경주맛집',
 '#요리',
 '#맛집',
 '#맛스타그램']

# 데이터저장

| 검색키워드 | 이미지파일명 | 해시태그 |
| --- | --- | --- |
| 음식스타그램 | asdiim23mmfs.jpg | #tag1 #tag2 ... |

이런식으로 하면 될라나

# 자동실행

In [131]:
img_name, img_url = get_image(browser, keyword='음식스타그램')
img_url

C:\Users\User\AppData\Local\Temp\ipykernel_16500\1405133385.py:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  img = browser.find_elements_by_class_name("KL4Bh")
C:\Users\User\anaconda3\envs\tfe\lib\site-packages\selenium\webdriver\remote\webelement.py:484: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


'https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/278533509_160944896318401_501511518362518509_n.webp?stp=dst-jpg_e35&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=101&_nc_ohc=BavFu8SYvnYAX9_qpRJ&edm=ABZsPhsBAAAA&ccb=7-4&ig_cache_key=MjgxODIxNDE2NDI1MTQ1Njg5NQ%3D%3D.2-ccb7-4&oh=00_AT_I1aZNs0UFDdUydsw3K6WzIYObA94qEFzQYfGBtr0MIg&oe=62624ADE&_nc_sid=4efc9f'

In [123]:
save_image(browser, '음식스타그램', img_name, img_url)

In [126]:
img_name

'622866a2-83c2-4609-a64b-7c1ae46deb8c.jpg'

In [132]:
out = get_hashtag(browser)

C:\Users\User\AppData\Local\Temp\ipykernel_16500\3343204787.py:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  plus = browser.find_elements_by_class_name("xil3i") #해쉬태그 클래스
C:\Users\User\AppData\Local\Temp\ipykernel_16500\3343204787.py:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  plus = browser.find_elements_by_class_name("xil3i")
C:\Users\User\AppData\Local\Temp\ipykernel_16500\3343204787.py:12: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  comments = browser.find_elements_by_class_name("EtaWk")


In [129]:
out

[]

In [15]:
browser = open_browser()
time.sleep(2)

login(browser)
time.sleep(3)

search_keyword(browser, keyword='음식스타그램')
time.sleep(10) #첫 검색은 상당히 오래 걸릴 수 있음.
# time.sleep(5 + (random() * 5))

open_first_post(browser)
time.sleep(5 + (random() * 5))
###### 여기서부터 게시물 넘겨가면서 반복 #####

likes = get_likes

C:\Users\User\AppData\Local\Temp\ipykernel_16500\1755654433.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome("chromedriver.exe") # 입력
C:\Users\User\AppData\Local\Temp\ipykernel_16500\1391365862.py:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  first_post = browser.find_elements_by_class_name("_9AhH0")[0]


<class 'selenium.webdriver.remote.webelement.WebElement'>
<selenium.webdriver.remote.webelement.WebElement (session="d60e783b00ebdf148df42c582934193a", element="96b970d6-eaa1-4ed7-bcd6-de7189c0565a")>
